In [1]:
import numpy as np
import astropy.coordinates as coord
import sys
import matplotlib.pyplot as plt
from astropy.table import Table,join

if './SelfCalGroupFinder/py/' not in sys.path:
    sys.path.append('./SelfCalGroupFinder/py/')
from groupcatalog import GroupCatalog, BGSGroupCatalog, TestGroupCatalog, serialize, deserialize, SDSSGroupCatalog
import catalog_definitions as cat
from pyutils import *
from dataloc import *
import plotting as pp
%load_ext autoreload
%autoreload 2

## Running Group Finder

This notebook will call the functions to preprocess, run group finder, and run postprocessing code to build up a GroupCatalog object, which mostly wraps a pandas DataFrame containing the resulting group catalog data.

After running this on a given GroupCatalog definition, a serialized (via pickle) version of the GroupCatalog object will exist which can be deserialized elsewhere for analysis. See post_plots.ipynb for that.

In [4]:
sdss_list : list[GroupCatalog] = [
    cat.sdss_vanilla,
    cat.sdss_colors,
    cat.sdss_colors_chi,
    cat.sdss_vanilla_v2,
    cat.sdss_colors_v2,
    cat.sdss_colors_chi_v2,
    cat.sdss_bgscut,
]
uchuu_list : list[GroupCatalog] = [
    cat.uchuu_all,
]
mxxl_list : list[GroupCatalog] = [
    cat.mxxl_all,
    #cat.mxxl_all_c,
    cat.mxxl_fiberonly,
    #cat.mxxl_fiberonly_c,
    cat.mxxl_nn,
    #cat.mxxl_nn_c,
    cat.mxxl_simple_2,
    #cat.mxxl_simple_2_c,
    cat.mxxl_simple_4,
    #cat.mxxl_simple_4_c,
]
bgs_sv3_list : list[GroupCatalog] = [
    cat.bgs_sv3_nn_10p,
    cat.bgs_sv3_nn_7p,
    cat.bgs_sv3_nn_6p,
    cat.bgs_sv3_fiberonly_10p,
    cat.bgs_sv3_simple_4_10p,
    cat.bgs_sv3_simple_4_9p,
    cat.bgs_sv3_simple_4_8p,
    cat.bgs_sv3_simple_4_7p,
    cat.bgs_sv3_simple_4_6p,
    cat.bgs_sv3_simple_4_5p,
    cat.bgs_sv3_simple_4_4p,
    cat.bgs_sv3_simple_4_3p,
    cat.bgs_sv3_simple_4_2p,
    cat.bgs_sv3_simple_4_1p,
    cat.bgs_sv3_simple_5_10p,
    cat.bgs_sv3_simple_5_9p,
    cat.bgs_sv3_simple_5_8p,
    cat.bgs_sv3_simple_5_7p,
    cat.bgs_sv3_pz_1_10p,
    cat.bgs_sv3_pz_1_0_7p,
    cat.bgs_sv3_pz_1_1_7p,
    cat.bgs_sv3_pz_1_2_7p,
    cat.bgs_sv3_pz_1_3_7p,
    cat.bgs_sv3_pz_2_0_7p
]
bgs_y1_list : list[GroupCatalog] = [
    #cat.bgs_simple_4_old,
    ##cat.bgs_simple_4,
    #cat.bgs_simple_4_1pass,
    #cat.bgs_simple_4_no_sdss,
    #cat.bgs_simple_4_4p,
    #cat.bgs_simple_4_c,
    #cat.bgs_fiberonly,
    #cat.bgs_fiberonly_1pass,
    #cat.bgs_nn,
    #cat.bgs_nn_sdsslike,
    ##cat.bgs_simple_2,
    #cat.bgs_simple_2_c,
    ##cat.bgs_simple_5,
]
bgs_y3_list : list[GroupCatalog] = [
    ##cat.bgs_y3_simple_4,
    #cat.bgs_y3_simple_4_4p,
    #cat.bgs_y3_fiberonly_1pass,
    #cat.bgs_y3_fiberonly,
    ##cat.bgs_y3_simple_5,
    cat.bgs_y3_pzp_1
]

datasets_to_run: list[GroupCatalog] = []
#datasets_to_run.extend(sdss_list)
#datasets_to_run.extend(uchuu_list)
#datasets_to_run.extend(mxxl_list)
#datasets_to_run.extend(bgs_sv3_list)  
#datasets_to_run.extend(bgs_y1_list)
#datasets_to_run.extend(bgs_y3_list)

#datasets_to_run.extend([cat.bgs_sv3_pz_2_0_7p])

#datasets_to_run.extend([
#    cat.bgs_sv3_pz_1_10p,
#    cat.bgs_sv3_pz_1_0_7p,
#    cat.bgs_sv3_pz_1_1_7p,
#    cat.bgs_sv3_pz_1_2_7p,
#    cat.bgs_sv3_pz_1_3_7p,
#   cat.bgs_sv3_pz_2_0_7p
#])

mcmc = BGSGroupCatalog("Photo-z Plus MCMC BGS sv3 7pass ", Mode.PHOTOZ_PLUS_v1, 19.5, 21.0, num_passes=10, drop_passes=3, data_cut='sv3', sdss_fill=False))
mcmc.GF_props = cat.GF_PROPS_VANILLA.copy()
datasets_to_run.extend([mcmc])

#datasets_to_run.extend([cat.bgs_sv3_pz_1_2_7p])

# To just run postprocessing on GF output, comment out run_group_finder()
for d in datasets_to_run:
    #d = deserialize(d)
    d.run_group_finder(popmock=False)
    d.postprocess()
    #d.run_corrfunc()
    serialize(d)
    #del(d)


Pre-processing...
Reading data from  /mount/sirocco1/imw2293/GROUP_CAT/DATA/BGS_FUJI/ian_BGS_SV3_merged.fits

Mode PHOTOZ PLUS
Using one set of extra parameter values for all color combinations.


ValueError: cannot convert float NaN to integer

In [70]:
n_range = [1, 20.99999] # we floor this value for the num neighbors to use
a_range = [0.0, 3.0]
b_range = [0.0, 5.0]
s_range = [0.00001, 0.1]
p_range = [1.0, 8.0]
n_walkers = 34

pos = np.array([np.random.uniform(low=[a_range[0], b_range[0], 10**(s_range[0]), p_range[0]], 
                                      high=[a_range[1], b_range[1], 10**(s_range[1]), p_range[1]]) for i in range(n_walkers*4)])
pos[:, 2] = np.log10(pos[:, 2])  # Convert back from log scale
pos = pos.reshape(n_walkers, 16)
pos = np.insert(pos, 0, np.arange(n_walkers)%20 +1, axis=1)
